In [33]:
from torch import tensor
from torch import nn 
from torch import sigmoid
import pandas as pd
import numpy as np
#import torch.nn.functional as F
#import torch.nn.functional
#import torch.nn.functional.sigmoid
import torch.optim as optim

df = pd.read_csv("/content/drive/My Drive/ANSYS/4th_dataset.csv")
df.head()

,Unnamed: 0,Iin[A],Iout[A],l[mm],p1[mm],p2[mm],p3[mm],win[mm],kdiff[%],Bleak[uT],V_PriWind[cm3],V_PriCore[cm3],Pout[W]
0,0,10,10,400,10,10,10,100,6.551479,2.9,38.16,800,50.312965
1,1,10,20,400,10,10,10,100,6.551479,5.5,38.16,800,100.625931
2,2,10,40,400,10,10,10,100,6.551479,10.7,38.16,800,201.251862
3,3,20,10,400,10,10,10,100,6.551479,3.2,38.16,800,50.312965
4,4,20,20,400,10,10,10,100,6.551479,5.8,38.16,800,100.625931


In [34]:
x_df = df.loc[:, 'kdiff[%]':'Pout[W]']
y_df = df.loc[:, 'Iin[A]':'win[mm]']
 

y_df.head()

,Iin[A],Iout[A],l[mm],p1[mm],p2[mm],p3[mm],win[mm]
0,10,10,400,10,10,10,100
1,10,20,400,10,10,10,100
2,10,40,400,10,10,10,100
3,20,10,400,10,10,10,100
4,20,20,400,10,10,10,100


In [74]:
import torch
#device
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x_np = x_df.values
y_np = y_df.values
import numpy as np
def zscore(x, axis = 0):
    xmean = x.mean(axis=axis, keepdims=True)
    xstd  = np.std(x, axis=axis, keepdims=True)
    zscore = (x-xmean)/xstd
    return zscore

def min_max(x, axis=0):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result

#x_np = zscore(x_np)
#y_np = zscore(y_np)
x_np = min_max(x_np)
y_np = min_max(y_np)

x_tensor=torch.tensor(x_np)
y_tensor=torch.tensor(y_np)

x_data = x_tensor.to(device)
x_data = x_tensor.to(dtype)
y_data = y_tensor.to(device)
y_data = y_tensor.to(dtype)

print(x_data.dtype)
print(y_data.dtype)


torch.float32
torch.float32


In [75]:
x_np

array([[0.36835365, 0.        , 0.        , 0.        , 0.        ],
       [0.36835365, 0.01725282, 0.        , 0.        , 0.02622281],
       [0.36835365, 0.05175846, 0.        , 0.        , 0.07866844],
       ...,
       [0.02894263, 0.24414227, 1.        , 0.21568627, 0.0676139 ],
       [0.02894263, 0.36612064, 1.        , 0.21568627, 0.16145061],
       [0.02894263, 0.3863217 , 1.        , 0.21568627, 0.34912403]])

In [79]:
#x_data = tensor([[1.0], [2.0], [3.0], [4.0]]) # input data
#y_data = tensor([[0.], [0.], [1.], [1.]]) # true output

# STEP 1: Design your model using class
class Model(nn.Module):
  def __init__(self):
    #In the constructor, we instantiate nn.Linear module.
    super(Model, self).__init__()
    self.linear = nn.Linear(5, 7) # nn.Linear(<input size> ,<output size>)

  def forward(self, x):
    y_pred = sigmoid(self.linear(x))
    #y_pred = F.sigmoid(self.linear(x)) # Either programs work.
    return y_pred

# our model
model=Model()

# STEP 2: Construct loss and optimizer
# Construct our loss function and an Optimizer. The call to model. parameters()
# in the SGD constructor will contain the learnable parameters of the two 
# nn.Linear moduled which are members of the model.

#criterion = nn.BCELoss(reduction='mean') # BCELoss: Binary Cross Entropy Loss
criterion = nn.BCELoss(size_average=True) # Either programs work.
#size_avarage define whethere devide by bach-size or not.
#criterion = torch.nn.BCELoss(size_average=False) # 
#criterion = torch.nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=1)
# STEP 3: Training cycle (forward, backward, update)
for epoch in range(1000):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x_data)

  # Compute and print loss
  loss = criterion(y_pred, y_data)
  print(f'Epoch {epoch + 1}/1000 | Loss: {loss.item(): .4f}')

  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Epoch 1/1000 | Loss:  0.6893
Epoch 2/1000 | Loss:  0.6868
Epoch 3/1000 | Loss:  0.6846
Epoch 4/1000 | Loss:  0.6825
Epoch 5/1000 | Loss:  0.6807
Epoch 6/1000 | Loss:  0.6790
Epoch 7/1000 | Loss:  0.6774
Epoch 8/1000 | Loss:  0.6760
Epoch 9/1000 | Loss:  0.6747
Epoch 10/1000 | Loss:  0.6734
Epoch 11/1000 | Loss:  0.6723
Epoch 12/1000 | Loss:  0.6713
Epoch 13/1000 | Loss:  0.6703
Epoch 14/1000 | Loss:  0.6694
Epoch 15/1000 | Loss:  0.6685
Epoch 16/1000 | Loss:  0.6677
Epoch 17/1000 | Loss:  0.6669
Epoch 18/1000 | Loss:  0.6662
Epoch 19/1000 | Loss:  0.6655
Epoch 20/1000 | Loss:  0.6648
Epoch 21/1000 | Loss:  0.6641
Epoch 22/1000 | Loss:  0.6635
Epoch 23/1000 | Loss:  0.6629
Epoch 24/1000 | Loss:  0.6623
Epoch 25/1000 | Loss:  0.6617
Epoch 26/1000 | Loss:  0.6611
Epoch 27/1000 | Loss:  0.6606
Epoch 28/1000 | Loss:  0.6601
Epoch 29/1000 | Loss:  0.6595
Epoch 30/1000 | Loss:  0.6590
Epoch 31/1000 | Loss:  0.6585
Epoch 32/1000 | Loss:  0.6580
Epoch 33/1000 | Loss:  0.6575
Epoch 34/1000 | Los

In [0]:



# After training
#print(f'\nLet\'s predict the hours need to score above 50%\n{"=" * 50}')
#hour_var = model(tensor([[1.0]]))
#print(f'Prediction after 1 hour of training: {hour_var.item(): .4f} | Above 50%: {hour_var.item() > 0.5}' )
#hour_var = model(tensor([[7.0]]))
#print(f'Prediction after 7 hours of training: {hour_var.item(): .4f} | Above 50%: {hour_var.item() > 0.5}')
